In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.43.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.39.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-telemetry 1.43.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-telemetry~=1.39.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.43.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from

In [7]:
from obs.drift import execute
from datetime import datetime
from azure.ai.ml.constants import TimeZone
from dateutil import tz

from azure.ai.ml.entities import (
    CronSchedule,
    RecurrenceSchedule,
    RecurrencePattern,
    ScheduleStatus,
)
schedule_start_time = datetime.now(tz=tz.gettz("PACIFIC STANDARD TIME"))
# target_dt_step
cron_schedule = CronSchedule(
    expression="*/10 * * * *",
    start_time=schedule_start_time,
    time_zone=TimeZone.PACIFIC_STANDARD_TIME,
    status=ScheduleStatus.ENABLED,
)
base_table_name = "IRIS_DATA_NEW"
target_table_name = base_table_name
base_dt_from = "12/15/2021"
base_dt_to = "03/01/2022"
target_dt_from = "04/01/2022"
target_dt_to ="05/13/2022"
target_dt_shift_step_size= "D"

ml_client, job_name = execute(subscription_id="0e9bace8-7a81-4922-83b5-d995ff706507",resource_group="azureml",workspace="ws01ent", compute_name ='DS11', experiment_name= "drift-analysis-job", base_table_name =base_table_name, drift_result_table="drift_analysis_result",
target_table_name =base_table_name, base_dt_from =base_dt_from, base_dt_to= base_dt_to,target_dt_from=target_dt_from, target_dt_to=target_dt_to, user_defined_module_file="drift_job/drift_analysis_user.py", user_defined_conda_file="drift_job/conda.yml", 
drift_analysis_job_file="drift_job/drift_analysis_aml_job.py",bin="7d", limit=100000, cron_schedule =None, concurrent_run=False, drift_threshold =0.4, target_dt_shift_step_size=target_dt_shift_step_size)



TypeError: execute() got an unexpected keyword argument 'target_dt_shift_step_size'

In [ ]:

ISDWeather_ISDWeather

In [69]:
from obs.query import KustoQuery
import numpy as np
from obs.drift.drift_analysis_kusto import Drift_Analysis_User
Drift_Analysis_User
kusto_query = Drift_Analysis_User(ws=ws)
drift_result_table="drift_analysis_result"
run_prefix="ISDWeather_ISDWeather_ddd"
last_run = kusto_query.query("let last_run_id = "+drift_result_table+"|where run_id like '"+run_prefix+"'| summarize max_run = max(run_id); "+drift_result_table+"|where run_id == toscalar(last_run_id)") 
print(last_run.shape[0])
last_target_date = last_run["target_end_date"][0]
print(last_target_date)
pd.date_range(last_target_date,periods=2).format()[-1]

0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [40]:
from obs.drift.drift_analysis_kusto import Drift_Analysis_User
from azureml.core import Workspace
from obs.drift import launch_dashboard, execute

ws = Workspace.from_config()
drift_analysis = Drift_Analysis_User(ws)


In [44]:
from importlib import reload
import logging
reload(logging)
logging.getLogger('werkzeug')

launch_dashboard(drift_analysis, port_num=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


In [ ]:
def cancel_job(ml_client, job_name):



    existing_job = ml_client.jobs.get(name=job_name)
    # change schedule status to disable
    existing_job.schedule.status = ScheduleStatus.DISABLED
    ml_client.jobs.create_or_update(existing_job, experiment_name=existing_job.experiment_name)
# cancel_job(ml_client, job_name)


In [ ]:
for job in ml_client.jobs.list():
    if job.display_name == "schedule_pipeline" and job.schedule.status!="disabled":
        cancel_job(ml_client, job.name)


In [ ]:
from aml_obs.query import RT_Visualization
# streaming_table_name="streaming_test"

# rt_viz =RT_Visualization(streaming_table_name,ws)
# rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Anomaly Detection

In [ ]:
from aml_obs.query import KustoQuery
query = KustoQuery(streaming_table_name, ws)

query.anomaly_detection(min_t="4/26/2022 10:08:45", max_t="4/23/2022 10:10:00", step="0.5s", metric="metric1", agg="avg", ts_col="timestamp", groupby="lr", sensitivity=0.1, filter="0.001")


### Drift Query

In [ ]:

from obs.drift.drift_analysis_kusto import Drift_Analysis_User

In [ ]:
drift_analysis =Drift_Analysis_User(ws)
#test1 1m 1m28 vs 2m26
#test2 10m 1459 vs 10m59


In [ ]:
drift_analysis.sample_data_points("StormEvents", col_name="EventId", start_date = "01/01/2007", horizon=30 )

In [ ]:
df_output, drift_result= drift_analysis.analyze_drift(limit=10000,base_table_name = 'ISDWeather',target_table_name='ISDWeather', base_dt_from='2013-04-13', base_dt_to='2014-05-13', target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')


In [ ]:
drift_result

In [ ]:
feature_distribution = drift_analysis.get_features_distributions(target_table_name='ISDWeather',target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')

In [ ]:
from aml_obs.drift import execute_drift_detect_job
execute_drift_detect_job(subscription_id="0e9bace8-7a81-4922-83b5-d995ff706507",resource_group="azureml",workspace="ws01ent", compute_name ='DS11', experiment_name= "drift-analysis-job", base_table_name ="ISDWeather", 
target_table_name ="ISDWeather", base_dt_from ="2013-05-13", base_dt_to= "2014-06-13",target_dt_from="2013-04-13", target_dt_to="2014-05-13", bin="7d", limit=3000000)

In [ ]:
# #Option to set parameters to a custom ADX cluster
# from azure.mgmt.kusto import KustoManagementClient
# from azure.mgmt.kusto.models import Cluster, AzureSku
# from azure.common.credentials import ServicePrincipalCredentials
# from azure.mgmt.kusto import KustoManagementClient
# from azure.mgmt.kusto.models import Cluster, AzureSku,LanguageExtension, LanguageExtensionsList
# from azure.mgmt.kusto.models import ReadWriteDatabase
# from azure.mgmt.kusto.models import DatabasePrincipalAssignment
# from azure.identity import ClientSecretCredential 


# tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# #Application ID
# client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
# #Client Secret, set it at your WS' keyvault with key name same as your client_id
# client_secret = kv.get_secret(client_id)
# subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

# cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster
# # kv.set_secret(KV_SP_ID,client_id)
# # kv.set_secret(KV_SP_KEY,client_secret)
# # kv.set_secret(KV_ADX_DB,"db01")
# # kv.set_secret(KV_ADX_URI,cluster_uri)
# # kv.set_secret(KV_TENANT_ID,tenant_id)

# credentials = ClientSecretCredential(
#     client_id=client_id,
#     client_secret=client_secret,
#     tenant_id=tenant_id
# )
# cluster_name = "adx02"
# resource_group_name = "databackend"
# kusto_management_client = KustoManagementClient(credentials, subscription_id)

# cluster_operations = kusto_management_client.clusters





In [ ]:
# poller = cluster_operations.begin_add_language_extensions(resource_group_name, cluster_name, LanguageExtensionsList(value=[LanguageExtension(language_extension_name="PYTHON")]))
# poller.wait()

In [2]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
# prov_config = AksCompute.provisioning_configuration()



aks_name = 'aks002'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

InProgress........................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
